### Prueba con codigo hecho con chatgpt

In [2]:
import os
import pandas as pd
import shutil

In [4]:
# Define destination path and name of the street
street_name = "quimey_co"                                                          # Define name
destination_folder = "/mnt/d/My_Prj/cavi_prj/dataDynamo/"                           # Define path
destination_path = os.path.join(destination_folder,"trazaxyz_"+ street_name + ".csv")

# Process the copied trazaxyz_street.csv file
def clean_line(line):
    line = line.replace("Point(X = ", "").replace(" Y = ", "").replace(" Z = ", "").replace(")", "")
    return line

# Read the trazaxyz_street.csv , clean it, and save to Extracted_Coordinates.csv (renaming if necessary)
cleaned_lines = []
with open(destination_path, 'r') as file:
    for line in file:
        cleaned_lines.append(clean_line(line))

# Ensure the cleaned file is saved with a unique name if it already exists
output_file_path = os.path.join(destination_folder, f'Extracted_Coordinates_{street_name}.csv')
if os.path.exists(output_file_path):
    counter = 1
    while os.path.exists(output_file_path):
        output_file_path = os.path.join(destination_folder, f'Extracted_Coordinates_{street_name}_{counter}.csv')
        counter += 1

# Write the cleaned lines to the new CSV file
with open(output_file_path, 'w') as file:
    file.writelines(cleaned_lines)

# Load the cleaned coordinates CSV file
coordinates_data = pd.read_csv(output_file_path, header=None)
coordinates_data.columns = ['X', 'Y', 'Z']

# Load the SWMM model file
swmm_model_file_path = 'A-12-CAV-SWMM-Actual_test.inp'                          # Define source file location
with open(swmm_model_file_path, 'r', encoding='latin1') as file:
    swmm_model_data = file.readlines()

# Generate new junction names and prepare data to add
new_junctions = []
new_coordinates = []
junction_counter = 1

for index, row in coordinates_data.iterrows():
    junction_name = f"{street_name}_{junction_counter}"
    x, y, z = row['X'], row['Y'], row['Z']
    
    # Add to [COORDINATES] section
    new_coordinates.append(f"{junction_name}\t{x:.3f}\t{y:.3f}\n")
    
    # Add to [JUNCTIONS] section
    new_junctions.append(f"{junction_name}\t{z:.3f}\t0\t0\t0\t0\n")
    
    junction_counter += 1

# Find where to insert the new data in the SWMM model file
coordinates_section_start = None
junctions_section_start = None

for i, line in enumerate(swmm_model_data):
    if '[COORDINATES]' in line:
        coordinates_section_start = i + 3  # Skip the header line
    if '[JUNCTIONS]' in line:
        junctions_section_start = i + 3  # Skip the header line

# Insert new junction data
if coordinates_section_start is not None:
    swmm_model_data[coordinates_section_start:coordinates_section_start] = new_coordinates

if junctions_section_start is not None:
    swmm_model_data[junctions_section_start:junctions_section_start] = new_junctions

# Save the modified SWMM model file
modified_swmm_model_file_path = 'Modified_A-12-CAV-SWMM-Actual_test.inp'        # Define path
with open(modified_swmm_model_file_path, 'w', encoding='latin1') as file:
    file.writelines(swmm_model_data)

print(f"Processed coordinates file saved to: {output_file_path}")
print(f"Modified file saved to: {modified_swmm_model_file_path}")

Processed coordinates file saved to: /mnt/d/My_Prj/cavi_prj/dataDynamo/Extracted_Coordinates_quimey_co.csv
Modified file saved to: Modified_A-12-CAV-SWMM-Actual_test.inp
